From https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

batches = 0
for X, y in test_dataloader:
    if batches == 0:
        print(f"Shape of X [N, C, H, W]: {X.shape}")
        print(f"Shape of y: {y.shape} {y.dtype}")
    batches += 1

print(f'{batches} in test db')

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
157 in test db


In [4]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

model = NeuralNetwork().to(device)
print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=2e-3)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # show results every 100 batches
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            accuracy = 100.0 * (pred.argmax(1) == y).type(torch.float).sum().item() / len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}] accuracy: {accuracy:>0.1f}%")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Epoch 1
-------------------------------
loss: 0.242442  [    0/60000] accuracy: 93.8%
loss: 0.392971  [ 6400/60000] accuracy: 85.9%
loss: 0.242672  [12800/60000] accuracy: 90.6%
loss: 0.424221  [19200/60000] accuracy: 78.1%
loss: 0.334810  [25600/60000] accuracy: 85.9%
loss: 0.395811  [32000/60000] accuracy: 82.8%
loss: 0.380992  [38400/60000] accuracy: 89.1%
loss: 0.548841  [44800/60000] accuracy: 79.7%
loss: 0.485027  [51200/60000] accuracy: 84.4%
loss: 0.368610  [57600/60000] accuracy: 85.9%
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.420936 

Epoch 2
-------------------------------
loss: 0.241452  [    0/60000] accuracy: 93.8%
loss: 0.391041  [ 6400/60000] accuracy: 87.5%
loss: 0.241673  [12800/60000] accuracy: 89.1%
loss: 0.422261  [19200/60000] accuracy: 78.1%
loss: 0.333010  [25600/60000] accuracy: 85.9%
loss: 0.394113  [32000/60000] accuracy: 82.8%
loss: 0.379359  [38400/60000] accuracy: 89.1%
loss: 0.546794  [44800/60000] accuracy: 79.7%
loss: 0.483122  [51200/60000] accuracy: 

In [8]:
loaded_model = NeuralNetwork()
loaded_model.load_state_dict(torch.load("model.pth"))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

loaded_model.eval()
correct = 0
sample = 100
for i in range(sample):
    x, y = test_data[i][0], test_data[i][1]
    with torch.no_grad():
        pred = loaded_model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')
        if predicted == actual:
            correct += 1
print(f'accuracy: {correct/sample*100.0}%')

Predicted: "Ankle boot", Actual: "Ankle boot"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Shirt", Actual: "Shirt"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Coat", Actual: "Coat"
Predicted: "Shirt", Actual: "Shirt"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"
Predicted: "Pullover", Actual: "Coat"
Predicted: "Sandal", Actual: "Sandal"
Predicted: "Sandal", Actual: "Sneaker"
Predicted: "Dress", Actual: "Dress"
Predicted: "Coat", Actual: "Coat"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Shirt", Actual: "Coat"
Predicted: "Bag", Actual: "Bag"
Predicted: "T-shirt/top", Actual: "T-shirt/top"
Predicted: "T-shirt/top", Actual: "Pullover"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"
Predicted: "Sandal", Actual: "Ankle boot"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Pullove